# About

For an EdTech company, randomly approaching universities may not result in satisfying new customers relative to the number of leads (i.e., close/rate ratio). I outlined three approaches using IPEDS data that an EdTech company can consider to improve its close/rate ratio. The term *member* refers to *customer* throughout this notebook.

In [ ]:
import pandas as pd
import membership

# Preprocess

In [ ]:
# load ids of current members
file_name_members = 'members_ipeds_ids.txt'
ids_members = membership.gen_ids_members(file_name_members)

In [ ]:
# create ids of all universities that meet criteria for the company's membership
file_name_ipeds = 'data_ipeds.csv'
ids_all = membership.gen_ids_all(file_name_ipeds)

In [ ]:
# create ids of non-members
ids_non_members = membership.gen_ids_nonmembers(ids_all, ids_members)

# Current market share

In [ ]:
# compute market share
company_market_share = round(membership.compute_market_share(ids_all, ids_members))
print("The EdTech company's market share is {} percent".format(company_market_share))

# Approach 1: Need base identification

This framework aims to generate a need score for each non-member university and select those with high need scores as potential members. Need scores are generated based on criteria related to the university's needs, such as expanding college access and building a more diverse cohort of students. Upon sorting non-member universities based on their need scores, the company marketing team can decide on the number of potential members as much as the capacity of their sales or customer relation team allows.

I used three needs criteria in this notebook: access expansion, student diversity, and streamlining the admission process. The metric for expanding access was whether the university offers distance education. Two metrics were implemented to capture the needs for student diversity: diversity of gender and diversity of race of undergraduate students enrolled at the university. The metric indicating the need for streamlining the admission process was the number of undergraduate students enrolled. Ideally, we would use the number of undergraduate applications received by the university rather than the number of students enrolled. The more applications that a university admission department works with, the more likely it is to need a software system that helps them streamline the admission process. A list of 100 non-members with the highest need scores was generated in the sample code. The marketing team can customize the number of non-members in the list based on the sales team's capacity.

This prototype can be extended in at least three ways. Firstly, one can add criteria as well as metrics. For example, one can include the number of university programs, the diversity of financial aids, and trend-related metrics. Secondly, my sample code uses equal weighting for each metric. An extension could be an interactive dashboard in which the EdTech company’s marketing team can enter different weights based on their knowledge of the current state of the education industry. Thirdly, related to the interactivity noted earlier, a dashboard where the marketing team can pick and choose criteria and metrics for generating need scores could be useful. For example, the marketing team may use a different set of diversity metrics for universities that only accept one gender (i.e., women's colleges). The gender diversity metric for these universities would naturally be zero.

In [ ]:
# subset ipeds data for non-members

var_list = [
    'UnitID','Institution Name',
    'City location of institution (HD2020)',
     'State abbreviation (HD2020)',
     'ZIP code (HD2020)',
    'Undergraduate level programs or courses are offered via distance education (IC2020)',
    'Percent of full-time first-time undergraduates awarded any financial aid (SFA1920)',
    'Total men (EF2020  All students  Undergraduate total)',
    'Total women (EF2020  All students  Undergraduate total)',
    'American Indian or Alaska Native total (EF2020A  All students  Undergraduate total)',
    'Asian total (EF2020A  All students  Undergraduate total)',
    'Black or African American total (EF2020A  All students  Undergraduate total)',
    'Hispanic total (EF2020A  All students  Undergraduate total)',
    'Native Hawaiian or Other Pacific Islander total (EF2020A  All students  Undergraduate total)',
    'White total (EF2020A  All students  Undergraduate total)',
    'Grand total (EF2020  All students  Undergraduate total)'
]

subset_df = membership.subset(file_name_ipeds, ids_non_members, var_list)

### Create diversity metrics

In [ ]:
# create diversity metrics: gender
col_dict_gender = {
    'Total men (EF2020  All students  Undergraduate total)': 'p_men',
    'Total women (EF2020  All students  Undergraduate total)' : 'p_women'
}

prop_list_gender = list(col_dict_gender.values())

subset_df = membership.gen_proportions(subset_df, col_dict_gender)
subset_df['diversity_gender'] = membership.gen_blau_index(subset_df, prop_list_gender)

In [ ]:
# create diversity metrics: race
col_dict_race = {
    'American Indian or Alaska Native total (EF2020A  All students  Undergraduate total)': 'p_native_american',
    'Asian total (EF2020A  All students  Undergraduate total)' : 'p_asian',
    'Black or African American total (EF2020A  All students  Undergraduate total)': 'p_black',
    'Hispanic total (EF2020A  All students  Undergraduate total)': 'p_hispanic',
    'Native Hawaiian or Other Pacific Islander total (EF2020A  All students  Undergraduate total)':'p_hawaiian_pacific',
    'White total (EF2020A  All students  Undergraduate total)': 'p_white'
}

prop_list_race = list(col_dict_race.values())

subset_df = membership.gen_proportions(subset_df, col_dict_race)
subset_df['diversity_race'] = membership.gen_blau_index(subset_df, prop_list_race)

### Calculate need scores

In [ ]:
# a dictionary of variable to standardized
standardized_dict = {
    'diversity_gender' : 'scaled_diversity_gender',
    'diversity_race' : 'scaled_diversity_race',
    'Percent of full-time first-time undergraduates awarded any financial aid (SFA1920)' : 'scaled_financial_aid',
    'Undergraduate level programs or courses are offered via distance education (IC2020)' : 'scaled_distance_educ',
    'Grand total (EF2020  All students  Undergraduate total)' : 'scaled_total_ug'
}

# a list of variables which signs will be flipped (see membership.py for details)
vars_flipped = ['scaled_diversity_gender', 'scaled_diversity_race']

In [ ]:
# calculate need score
subset_df = membership.compute_need_scores(subset_df, standardized_dict, vars_flipped)

In [ ]:
# keep only relevant variables
vars_to_keep = [
    'UnitID','Institution Name',
    'City location of institution (HD2020)',
     'State abbreviation (HD2020)',
    'need_score'
]

need_scores_df = subset_df[vars_to_keep]

### Identify potential members
In this approach, the more fitting non-members are those with higher need scores.

In [ ]:
# select top n universities with highest need scores
n = 100
potential_members = need_scores_df.sort_values('need_score', ascending=False).head(n)

In [ ]:
# the first 10 universities in the list of potential members
potential_members.head(10)

In [ ]:
# to export to a csv file uncomment and run the line below
#potential_members.to_csv('potential_members_need_scores.csv', encoding='utf-8', index = False)

# Approach 2: Level of market penetration

The idea underlying this framework is to generate a market penetration score – that is, the number of member universities to the number of total universities in the area – and select non-member universities in regions that are low in market penetration. Then, after sorting regions based on the market penetration score, the EdTech company’s marketing team can choose the number of potential members according to the capacity of their sales team in regions that the EdTech company has a low level of market penetration. 

I used a state to define a region in this notebook. Using this approach, we can see states where the company has low or no market presence. A possible extension of this sample code is choosing a different unit area, for example, city or county.

In [ ]:
# subset ipeds data for all qualifying universities
var_list = ['UnitID', 'Institution Name',
            'City location of institution (HD2020)',
            'State abbreviation (HD2020)']

subset_df = membership.subset(file_name_ipeds, ids_all, var_list)

In [ ]:
# create a data frame of membership status
ids_status = membership.tag_membership(ids_all, ids_members)

In [ ]:
# merge and rename
subset_df = subset_df.merge(ids_status, on='UnitID', how='inner')

subset_df = subset_df.rename(columns={'City location of institution (HD2020)':'city',
                                       'State abbreviation (HD2020)' : 'state'})

In [ ]:
# preprocess variables
market_df= subset_df.groupby(['state','membership_status'], as_index = False).agg({'UnitID':'count'}).sort_values(
by = 'state')
market_df = market_df.pivot_table('UnitID', index='state', columns='membership_status', fill_value=0, aggfunc='sum')
market_df = market_df.reset_index()
market_df.columns.names = ['']

In [ ]:
# calculate market penetration score
market_df['total'] = market_df['member'] + market_df['non_member']
market_df['market_penetration_score'] = market_df['member']/market_df['total']

In [ ]:
# sort by market penetration score from lowest to highest
market_df = market_df.sort_values(by='market_penetration_score')

In [ ]:
# the first 15 states with lowest market penetration score
market_df.head(15)

In [ ]:
# to export to a csv file uncomment and run the line below
#market_df.to_csv('market_penetration_scores_by_states.csv', encoding='utf-8', index = False)

# Approach 3: Leverage the characteristics of current members

The idea in this framework is that characteristics of current members are projected to identify potential members. Thus, potential members are likely to have similar characteristics as current members.